In [ ]:
import tensorflow as tf
import numpy as np
import os,sys
import matplotlib.pyplot as plt
import scipy.io as scio
import numpy as np
import tensorflow.keras.optimizers as optimizers
import keras

from keras import Input, regularizers
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneGroupOut,LeavePGroupsOut
from keras.layers import Flatten,Dense,Dropout
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)#允许GPU分配的内存随程序需求而增长

print("Part One,Done")


# 数据读取及整合

In [ ]:
#文件读取Step1
path='F://data//FAIMS_rats//rat_data//data4//'#path=数据集存放位置
Control_File =path+'RAT_Control_Pus_48h_9'#RAT_Control_Pus_48h_9
Ecoli_File =path+ 'RAT_Ecoli_Pus_48h_12'
Parug_File = path+'RAT_Parug_Pus_48h_10'
Saureus_File = path+'RAT_Saureus_Pus_48h_10'

#文件读取Step2
Control = scio.loadmat(Control_File)
Ecoli = scio.loadmat(Ecoli_File)
Parug = scio.loadmat(Parug_File)
Saureus = scio.loadmat(Saureus_File)
#print(Control.keys())

#感染数据获取
data_Control =Control.get('Control_Pus_48h_9')
data_Ecoli =Ecoli.get('Ecoli_Pus_48h_12')
data_Parug =Parug.get('Parug_Pus_48h_10')
data_Saureus =Saureus.get('Saureus_Pus_48h_10')

#小鼠测试样本数读取
RatNum_Control=Control.get('Control_Pus_48h_9_RATTUnum')
RatNum_Ecoli=Ecoli.get('Ecoli_Pus_48h_12_RATTUnum')
RatNum_Parug=Parug.get('Parug_Pus_48h_10_RATTUnum')
RatNum_Saureus=Saureus.get('Saureus_Pus_48h_10_RATTUnum')
print(len(RatNum_Control))
'''
[[18 15 18 17 18 21 17 20 19]]
[[17 20 18 17 13 17 17 20 18 20 16 19]]
[[18 12 23 17 10 18 16 17 19 24]]
[[18 26 17 21 24 20 22 19 19 17]]
'''
RatNum_Control = RatNum_Control.reshape(-1)#降维 for 后续操作
RatNum_Ecoli = RatNum_Ecoli.reshape(-1)
RatNum_Parug = RatNum_Parug.reshape(-1)
RatNum_Saureus = RatNum_Saureus.reshape(-1)

#感染标签读取
label_Control =Control.get('Control_Pus_48h_9_LABEL')
label_Ecoli   =Ecoli.get('Ecoli_Pus_48h_12_LABEL')
label_Parug  =Parug.get('Parug_Pus_48h_10_LABEL')
label_Saureus =Saureus.get('Saureus_Pus_48h_10_LABEL')
print(label_Control.shape)
print(label_Ecoli.shape)
print(label_Parug.shape)
print(label_Saureus.shape)
print("This is label\n\n")
#感染数据替换for reduce class_num（感染标签虽是4个，但max=4，被判为5分类。所有感染标签值-1，max=3，为4分类）
label_Control =np.full(label_Control.shape,0)#181
label_Ecoli =np.full(label_Ecoli.shape,1)#173
label_Parug =np.full(label_Parug.shape,2)#177
label_Saureus =np.full(label_Saureus.shape,3)#187


#老鼠标签
label_rat_Control =Control.get('Control_Pus_48h_9_RATLABEL')
label_rat_Ecoli   =Ecoli.get('Ecoli_Pus_48h_12_RATLABEL')
label_rat_Parug  =Parug.get('Parug_Pus_48h_10_RATLABEL')
label_rat_Saureus =Saureus.get('Saureus_Pus_48h_10_RATLABEL')
#老鼠标签整理for loo
label_rat_Control_added = np.full(label_rat_Control.shape,-1)
label_rat_Ecoli_added = np.full(label_rat_Ecoli.shape,8)
label_rat_Parug_added =np.full(label_rat_Parug.shape,20)
label_rat_Saureus_added =np.full(label_rat_Saureus.shape,30)

label_rat_Control = label_rat_Control +label_rat_Control_added
label_rat_Ecoli = label_rat_Ecoli +label_rat_Ecoli_added
label_rat_Parug = label_rat_Parug +label_rat_Parug_added
label_rat_Saureus = label_rat_Saureus +label_rat_Saureus_added
#print(label_rat_Parug)

#数据、感染标签、小鼠标签整和
total_data_combat =np.concatenate([data_Control,data_Ecoli,data_Parug,data_Saureus],axis=2)#(102, 512, 718)
total_label_combat =np.concatenate([label_Control,label_Ecoli,label_Parug,label_Saureus])#(718,1)
total_label_rat_combat =np.concatenate([label_rat_Control,label_rat_Ecoli,label_rat_Parug,label_rat_Saureus])#(718,1)
total_rat_num = np.concatenate([RatNum_Control,RatNum_Ecoli,RatNum_Parug,RatNum_Saureus],axis=0).reshape(41)

print(total_data_combat.shape)
print(total_label_rat_combat.shape)
print(total_label_rat_combat.shape)
print("Part Two,Done")


# 统一老鼠的样本个数

In [ ]:
##冗余样本删除

#记录每个老鼠出现次数的数组
rat_times=np.zeros(41)

#冗余样本位置获取
redundant=np.zeros(752)
j=0
for i in range(752):
    rat_this = total_label_rat_combat[i]
    rat_times[rat_this]+=1
    if rat_times[rat_this]>15:
        redundant[j]=i
        j+=1
redundant = np.unique(redundant,axis = 0)
redundant =np.delete(redundant,[0],axis=0)
#print(redundant)#冗余样本位置
#删除
total_data_EP_ori=np.delete(total_data_combat,redundant,axis=2)
total_label_EP_ori=np.delete(total_label_combat,redundant,axis=0)
total_label_rat_EP_ori=np.delete(total_label_rat_combat,redundant,axis=0)
#print(total_data_EP_ori.shape,total_label_EP_ori.shape)#冗余样本删除后的数组


##缺少样本补足

#data数组补足
data_patch_EP_13 = total_data_EP_ori[:,:,206:208]
data_patch_EP_12 = total_data_EP_ori[:,:,337:340]
data_patch_EP_10 = total_data_EP_ori[:,:,375:380]

total_data_EP_13_1=total_data_EP_ori[:,:,0:208]
total_data_EP_13_2=total_data_EP_ori[:,:,208:]
total_data_EP_13= np.concatenate([total_data_EP_13_1,data_patch_EP_13,total_data_EP_13_2],axis=2)

total_data_EP_12_1=total_data_EP_13[:,:,0:342]
total_data_EP_12_2=total_data_EP_13[:,:,342:]
total_data_EP_12= np.concatenate([total_data_EP_12_1,data_patch_EP_12,total_data_EP_12_2],axis=2)

total_data_EP_10_1=total_data_EP_12[:,:,0:385]
total_data_EP_10_2=total_data_EP_12[:,:,385:]
total_data_EP_10= np.concatenate([total_data_EP_10_1,data_patch_EP_10,total_data_EP_10_2],axis=2)
total_data_EP=total_data_EP_10

#label数组补足
label_patch_EP_13 = total_label_EP_ori[206:208]
label_patch_EP_12 = total_label_EP_ori[337:340]
label_patch_EP_10 = total_label_EP_ori[375:380]

total_label_EP_13_1=total_label_EP_ori[0:208]
total_label_EP_13_2=total_label_EP_ori[208:]
total_label_EP_13= np.concatenate([total_label_EP_13_1,label_patch_EP_13,total_label_EP_13_2],axis=0)

total_label_EP_12_1=total_label_EP_13[0:342]
total_label_EP_12_2=total_label_EP_13[342:]
total_label_EP_12= np.concatenate([total_label_EP_12_1,label_patch_EP_12,total_label_EP_12_2],axis=0)

total_label_EP_10_1=total_label_EP_12[0:385]
total_label_EP_10_2=total_label_EP_12[385:]
total_label_EP_10= np.concatenate([total_label_EP_10_1,label_patch_EP_10,total_label_EP_10_2],axis=0)
total_label_EP=total_label_EP_10

#老鼠样本个数补足
rat_patch_EP_13 = total_label_rat_EP_ori[206:208]
rat_patch_EP_12 = total_label_rat_EP_ori[337:340]
rat_patch_EP_10 = total_label_rat_EP_ori[375:380]

total_rat_EP_13_1=total_label_rat_EP_ori[0:208]
total_rat_EP_13_2=total_label_rat_EP_ori[208:]
total_rat_EP_13= np.concatenate([total_rat_EP_13_1,rat_patch_EP_13,total_rat_EP_13_2],axis=0)

total_rat_EP_12_1=total_rat_EP_13[0:342]
total_rat_EP_12_2=total_rat_EP_13[342:]
total_rat_EP_12= np.concatenate([total_rat_EP_12_1,rat_patch_EP_12,total_rat_EP_12_2],axis=0)

total_rat_EP_10_1=total_rat_EP_12[0:385]
total_rat_EP_10_2=total_rat_EP_12[385:]
total_rat_EP_10= np.concatenate([total_rat_EP_10_1,rat_patch_EP_10,total_rat_EP_10_2],axis=0)
total_rat_EP=total_rat_EP_10

# Size处理

In [ ]:
#维度处理（102,512,total_num）-->（total_num,102,512,1）
total_num = 615
total_data_change = np.swapaxes(total_data_EP,0,2)
total_data_change_change = np.swapaxes(total_data_change,1,2)#(718, 102, 512)
print(total_data_change_change.shape)
total_data_expend = np.expand_dims(total_data_change_change, axis=3)

#图片剪裁（total_num,102,512,1）-->(total_num,102,170,1)
total_data_cut = total_data_expend[:,:,150:320,:]
scaler = StandardScaler()
total_data_norm = scaler.fit_transform(total_data_cut.astype(np.float32).reshape(-1, 102*170)).reshape(-1, 102, 170,1)

#Final_data
total_data = np.concatenate([total_data_norm,total_data_norm,total_data_norm],axis=3)
total_label = total_label_EP.reshape(total_num)
total_label_rat = total_rat_EP.reshape(total_num)
print(total_data.shape)
print(total_label.shape)
print(total_label_rat.shape)
print("Part Three,Done")


# 模型编译

In [ ]:
def model_recompile():
    base_model = ResNet50(weights='imagenet', include_top=False,
                                      input_shape=(102, 170, 3), 
                                      pooling='max'
                           )#基础模型

    ResNet50_LAYERS_TO_FREEZE = 'res5b_branch2c'#模型冻结至该层    
    set_trainable = False
    for layer in base_model.layers:
        if layer.name == ResNet50_LAYERS_TO_FREEZE:
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False

    model =keras.Sequential()#使用Sequential()在resnet后添加网络层
    model.add(base_model)
    model.add(Dense(512,activation='relu', kernel_regularizer = regularizers.l2 (0.01)))
    model.add(Dense(4,activation='softmax'))
    model.compile(loss="sparse_categorical_crossentropy",
                    optimizer =keras.optimizers.RMSprop(lr=0.01, rho=0.9, epsilon=1e-06),
                    metrics =['accuracy'])
    return model

# 留一法

In [ ]:
#由于内存不足，一个子集的老鼠进行留一法需要分两次，分别保存为003,004.

fea_resnet = np.zeros(615)#保存提取出的特征
rat_now = 0#当前运行到第几只老鼠
lpgo = LeavePGroupsOut(n_groups=1)

for(train_index,test_index) in lpgo.split(total_data,groups=total_label_rat):
    if rat_now >=0 :#从第time只老鼠开始。（time_min==0）
        print("第",rat_now,"次留一法\n\n\n")
        train_num, test_num = train_index.shape, test_index.shape
        X_train, X_test = total_data[train_index], total_data[test_index]
        y_train, y_test = total_label[train_index],total_label[test_index]#训练集，测试集数据
        model =model_recompile()#模型重编译                      
        
        model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test),batch_size=8,verbose = 2)#模型训练
        features = model.predict(X_test,batch_size =3)#输出分类概率
        if(rat_now==0 or rat_now==20):
            fea_resnet=features
        else:
            fea_resnet=np.concatenate([fea_resnet,features])#ResNet深度特征整合
        print(features)
    rat_now+=1 
        #print(loss,accuracy)

print("model",1,"-----------------------------------------------------------\n\n\n")  

#ResNet深度特征本地保存
np.savetxt('003',fea_resnet)